## Intro
This is a simple ETL (extract, transform, load) job for Messier object data. The data are obtained from three sources:
1. The full Messier catalogue from http://www.klima-luft.de/steinicke/ngcic/ngcic_e.htm
1. Additional laymen data from Wikipedia
1. Images of the Messier objects from Wikipedia

The files are uploaded to GitHub as examples for the processing. Future ETL jobs with other DSOs, exoplanets, and eclipsing binaries will be handled without the examples.

In [3]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

### Extract

In [4]:
messier_df = pd.read_excel('raw_data/Historic_M.xls') 
cons_df = pd.read_excel('raw_data/list-constellations-677j.xls', usecols="B,C") 
data = pd.read_csv("raw_data/wiki_m.csv") 
add_df = data[["NGC/IC number", "Common name", "Object type"]]

In [5]:
html = urlopen('https://en.wikipedia.org/wiki/Messier_object')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img', {'src':re.compile('.jpg')})

### Transform

In [6]:
messier_df["name"] = "Messier " + messier_df["M"].map(str)
messier_df["params"] = "m" + messier_df["M"].map(str)
messier_df["ra"] = messier_df["RH"].map(str) + "h"+ messier_df["RM"].map(str) + "m" + messier_df["RS"].map(str) + "s"
messier_df["dec"] = messier_df["V"].map(str) + messier_df["DG"].map(str) + "d"+ messier_df["DM"].map(str) + "m" + messier_df["DS"].map(str) + "s"
del messier_df['M']
del messier_df['RH']
del messier_df['RM']
del messier_df['RS']
del messier_df['DG']
del messier_df['DM']
del messier_df['DS']
del messier_df['C']
del messier_df['N']
del messier_df['NI']
del messier_df['Class']
del messier_df['h']
del messier_df['GC']
del messier_df['Other']
del messier_df['Discoverer']
del messier_df['Y']
del messier_df['M.1']
del messier_df['D']
del messier_df['Ap']
del messier_df['T']
del messier_df['Observer1']
del messier_df['Observer2']
del messier_df['V']
del messier_df['Remarks']
del messier_df['S']
del messier_df['Type']
messier_df.rename({"Dist (kly)": "dist", "Vmag": "mag"}, axis=1, inplace=True)

messier_df.head()

,Con,dist,mag,name,params,ra,dec
0,TAU,6.5,8.4,Messier 1,m1,5h34m31.9s,+22d0m52s
1,AQR,33.0,6.6,Messier 2,m2,21h33m27.2s,-0d49m22s
2,CVN,33.9,6.3,Messier 3,m3,13h42m11.2s,+28d22m34s
3,SCO,7.2,5.4,Messier 4,m4,16h23m35.5s,-26d31m29s
4,SER,24.5,5.7,Messier 5,m5,15h18m33.8s,+2d5m0s


In [7]:
for i in range(len(messier_df["Con"])):
    for j in range(len(cons_df["Abbreviation"])):
        if messier_df["Con"][i] == cons_df["Abbreviation"][j].upper():
            messier_df["Con"][i] = cons_df["Constellation"][j]
            
messier_df.head()

/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/1800166425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messier_df["Con"][i] = cons_df["Constellation"][j]


,Con,dist,mag,name,params,ra,dec
0,Taurus,6.5,8.4,Messier 1,m1,5h34m31.9s,+22d0m52s
1,Aquarius,33.0,6.6,Messier 2,m2,21h33m27.2s,-0d49m22s
2,Canes Venatici,33.9,6.3,Messier 3,m3,13h42m11.2s,+28d22m34s
3,Scorpius,7.2,5.4,Messier 4,m4,16h23m35.5s,-26d31m29s
4,Serpens,24.5,5.7,Messier 5,m5,15h18m33.8s,+2d5m0s


In [10]:
check_galaxy = "galaxy"
check_emission = "H II"
check_asterism = "Star System"

for i in range(len(add_df["Object type"])):
    if add_df["Object type"][i].find(check_galaxy) != -1:
        add_df["Object type"][i] = "galaxy"
    if add_df["Object type"][i].find(check_emission) != -1:
        add_df["Object type"][i] = "emission nebula"
    if add_df["Object type"][i].find(check_asterism) != -1:
        add_df["Object type"][i] = "asterism"
    add_df["Object type"][i] = add_df["Object type"][i].lower()

/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/1188317650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df["Object type"][i] = add_df["Object type"][i].lower()
/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/1188317650.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df["Object type"][i] = "galaxy"


In [11]:
add_df["Common name"][16] = add_df["Common name"][16].replace(" or", "")
add_df["Common name"][32] = add_df["Common name"][32].replace("/", ", ")
add_df["Common name"][43] = add_df["Common name"][43].replace(" or", ",")
add_df["Common name"][73] = add_df["Common name"][73].replace("[91]", "")

add_df["Object type"][7] = add_df["Object type"][7].replace("Nebula with cluster", "emission nebula")
add_df["Object type"][23] = add_df["Object type"][23].replace("Milky Way star cloud", "star cloud")

add_df.head()

/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/2387462411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df["Common name"][16] = add_df["Common name"][16].replace(" or", "")
/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/2387462411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_df["Common name"][32] = add_df["Common name"][32].replace("/", ", ")
/var/folders/qp/9vkptzrj44z9f5y8mt7k96dr0000gn/T/ipykernel_73269/2387462411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

,NGC/IC number,Common name,Object type
0,NGC 1952,Crab Nebula,supernova remnant
1,NGC 7089,–,globular cluster
2,NGC 5272,–,globular cluster
3,NGC 6121,–,globular cluster
4,NGC 5904,–,globular cluster


In [12]:
full_messier = pd.concat([messier_df.reset_index(drop=True),add_df.reset_index(drop=True)],axis=1)
full_messier.rename({"NGC/IC number": "ngcic", "Common name": "common", "Object type": "type", "Con": "con"}, axis=1, inplace=True)
full_messier = full_messier[["params", "name", "ngcic", "common", "type", "ra", "dec", "mag", "dist", "con"]]
full_messier.head()

,params,name,ngcic,common,type,ra,dec,mag,dist,con
0,m1,Messier 1,NGC 1952,Crab Nebula,supernova remnant,5h34m31.9s,+22d0m52s,8.4,6.5,Taurus
1,m2,Messier 2,NGC 7089,–,globular cluster,21h33m27.2s,-0d49m22s,6.6,33.0,Aquarius
2,m3,Messier 3,NGC 5272,–,globular cluster,13h42m11.2s,+28d22m34s,6.3,33.9,Canes Venatici
3,m4,Messier 4,NGC 6121,–,globular cluster,16h23m35.5s,-26d31m29s,5.4,7.2,Scorpius
4,m5,Messier 5,NGC 5904,–,globular cluster,15h18m33.8s,+2d5m0s,5.7,24.5,Serpens


In [13]:
all_images = []

for image in images: 
    bigger_image = image['src'].replace("70px","500px").replace("//","")
    all_images.append("https://" + bigger_image)

all_images.pop()
all_images.pop(0)
all_images.pop(0)

all_images.insert(31,"https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/M32_Francione_inverted.jpg/200px-M32_Francione_inverted.jpg")
all_images.insert(56,"https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/M57_The_Ring_Nebula.JPG/200px-M57_The_Ring_Nebula.JPG")
all_images.insert(91,"https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/M92_Hubble_WikiSky.jpg/200px-M92_Hubble_WikiSky.jpg")
all_images.insert(101,"https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Ngc5866_hst_big.png/200px-Ngc5866_hst_big.png")

image_df = pd.DataFrame(all_images, columns =['image'])
image_df.head()

,image
0,https://upload.wikimedia.org/wikipedia/commons...
1,https://upload.wikimedia.org/wikipedia/commons...
2,https://upload.wikimedia.org/wikipedia/commons...
3,https://upload.wikimedia.org/wikipedia/commons...
4,https://upload.wikimedia.org/wikipedia/commons...


In [14]:
complete_messier = pd.concat([full_messier.reset_index(drop=True),image_df.reset_index(drop=True)],axis=1)
complete_messier.replace(',','', regex=True, inplace=True)
complete_messier

,params,name,ngcic,common,type,ra,dec,mag,dist,con,image
0,m1,Messier 1,NGC 1952,Crab Nebula,supernova remnant,5h34m31.9s,+22d0m52s,8.4,6.5000,Taurus,https://upload.wikimedia.org/wikipedia/commons...
1,m2,Messier 2,NGC 7089,–,globular cluster,21h33m27.2s,-0d49m22s,6.6,33.0000,Aquarius,https://upload.wikimedia.org/wikipedia/commons...
2,m3,Messier 3,NGC 5272,–,globular cluster,13h42m11.2s,+28d22m34s,6.3,33.9000,Canes Venatici,https://upload.wikimedia.org/wikipedia/commons...
3,m4,Messier 4,NGC 6121,–,globular cluster,16h23m35.5s,-26d31m29s,5.4,7.2000,Scorpius,https://upload.wikimedia.org/wikipedia/commons...
4,m5,Messier 5,NGC 5904,–,globular cluster,15h18m33.8s,+2d5m0s,5.7,24.5000,Serpens,https://upload.wikimedia.org/wikipedia/commons...
5,m6,Messier 6,NGC 6405,Butterfly Cluster,open cluster,17h40m20.0s,-32d15m30s,4.2,1.6000,Scorpius,https://upload.wikimedia.org/wikipedia/commons...
6,m7,Messier 7,NGC 6475,Ptolemy Cluster,open cluster,17h53m50.0s,-34d47m36s,3.3,0.9800,Scorpius,https://upload.wikimedia.org/wikipedia/commons...
7,m8,Messier 8,NGC 6523,Lagoon Nebula,emission nebula,18h3m42.0s,-24d22m48s,5.8,4.1000,Sagittarius,https://upload.wikimedia.org/wikipedia/commons...
8,m9,Messier 9,NGC 6333,–,globular cluster,17h19m11.8s,-18d30m57s,7.8,25.8000,Ophiuchus,https://upload.wikimedia.org/wikipedia/commons...
9,m10,Messier 10,NGC 6254,–,globular cluster,16h57m8.9s,-4d5m56s,6.6,14.3000,Ophiuchus,https://upload.wikimedia.org/wikipedia/commons...


### "Load"
Not a true load in the ETL process. The loading will be done in a separate Flask app instead.

In [15]:
complete_messier.to_csv('complete_messier.csv', index=False, header=False)

In [13]:
# complete_messier.to_csv('complete_messier_with_header.csv', index=False)